In [450]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier, StackingClassifier
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC

# read data

In [451]:
df_train = pd.read_csv('../datasets/train.csv')
df_test = pd.read_csv('../datasets/test.csv')

In [452]:
def convert_passengerID_2_index(df: pd.DataFrame):

    df = df.set_index("PassengerId")
    return df

def split_carbin(df: pd.DataFrame):

    for key, value in enumerate(['deck', 'num', 'side']):
        df[f'Carbin_{value}'] = df['Cabin'].apply(lambda x: str(x).split("/")[key] if len(str(x).split("/")) >= key+1 else None)

    df = df.drop('Cabin', axis=1)
    return df

def drop_columns(df: pd.DataFrame, columns:list):
    
    df = df.drop(columns=columns, axis=1)

    return df

In [453]:
df_train_cleansed = (
    df_train.pipe(convert_passengerID_2_index)
    .pipe(split_carbin)
    .pipe(drop_columns, ['Name', 'Carbin_num'])
)

In [454]:
df_test_cleansed = (
    df_test.pipe(convert_passengerID_2_index)
    .pipe(split_carbin)
    .pipe(drop_columns, ['Name', 'Carbin_num'])
)

# split data into X and y

In [455]:
def split_data_2_x_y(df:pd.DataFrame):

    columns = df.columns 
    X_columns = [i for i in columns if i != 'Transported']
    y_column = 'Transported' if 'Transported' in columns else None 

    if y_column:
        return df[X_columns], df[y_column]
    else:
        return df[X_columns], None

In [456]:
X, y = split_data_2_x_y(df_train_cleansed)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

In [457]:
X_test, _ = split_data_2_x_y(df_test_cleansed)

In [458]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Carbin_deck,Carbin_side
PassengerId,,,,,,,,,,,,
1510_01,Mars,False,TRAPPIST-1e,31.0,False,1226.0,0.0,1.0,0.0,0.0,F,S
7253_01,Europa,False,TRAPPIST-1e,26.0,False,0.0,896.0,0.0,690.0,1.0,D,P
4714_01,Earth,True,TRAPPIST-1e,24.0,False,0.0,0.0,0.0,0.0,0.0,G,P
7727_01,Earth,False,55 Cancri e,33.0,False,0.0,0.0,0.0,436.0,224.0,E,S
3237_01,Mars,False,TRAPPIST-1e,21.0,False,1097.0,0.0,80.0,589.0,0.0,D,P


# data pipeline

In [459]:
# create datapipeline functions

In [460]:
num_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="median")),
    ("standardize", StandardScaler())
])

cate_pipeline = Pipeline([
    ("Fill N/A value", SimpleImputer(strategy="most_frequent")),
    ("One Hot Encoder", OneHotEncoder(handle_unknown="ignore"))
])

In [461]:
num_attribs = X_train.select_dtypes(include="float").columns
cat_attribs = X_train.select_dtypes(include="object").columns

In [462]:
# preprocessing = ColumnTransformer([
#     ("num", num_pipeline, make_column_selector(dtype_include=np.number)),
#     ("cat", cate_pipeline, make_column_selector(dtype_include=object))
# ])

In [463]:
preprocessing = ColumnTransformer([
    ("num", num_pipeline, num_attribs),
    ("cat", cate_pipeline, cat_attribs)
])

In [464]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7823 entries, 1510_01 to 7775_01
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    7637 non-null   object 
 1   CryoSleep     7624 non-null   object 
 2   Destination   7658 non-null   object 
 3   Age           7659 non-null   float64
 4   VIP           7642 non-null   object 
 5   RoomService   7675 non-null   float64
 6   FoodCourt     7660 non-null   float64
 7   ShoppingMall  7637 non-null   float64
 8   Spa           7669 non-null   float64
 9   VRDeck        7654 non-null   float64
 10  Carbin_deck   7823 non-null   object 
 11  Carbin_side   7647 non-null   object 
dtypes: float64(6), object(6)
memory usage: 794.5+ KB


# Train the model

In [465]:
# Randome Forest Tree
rnd_clf = make_pipeline(preprocessing, RandomForestClassifier(random_state=42))
rnd_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                ('randomforestclassifier',
                 RandomForestClassifier(random_state=42))])

In [466]:
# SVC
svc_clf = make_pipeline(preprocessing, SVC(degree=4, probability=True, random_state=42))
svc_clf.fit(X_train, y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One Hot '
                                                                   'Encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                ('svc', SVC(degree=4, probability=True, random_state=42))])

# Tunning the model

### GridSearch CV - Randome Forest Tree

In [467]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("random_forest", RandomForestClassifier(random_state=42))
])

# param_grid = [
#     {'preprocessing__geo__n_clusters': [5, 8, 10],
#     'random_forest__max_features': [4, 6, 8]},
#     {'preprocessing__geo__n_clusters': [10, 15],
#      'random_forest__max_features': [6, 8, 10]}
# ]
param_grid = [
    {
    'random_forest__max_features': [6, 8, 10],
    'random_forest__n_estimators': [50, 100, 200, 500],
    'random_forest__max_depth': [5, 10, None],
    'random_forest__min_samples_split': [2, 5, 10],
    'random_forest__min_samples_leaf': [1, 2, 4],
    # 'random_forest__max_features': ['auto', 'sqrt']
    }
]
grid_search_rnd_clf = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search_rnd_clf.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(s...
                                                                         Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                                       ('random_forest',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid=[{'random_forest__max_depth': [5, 10, None],
                          'random_forest__max_features': [6, 8, 10],
                          'random_forest__min_samples_leaf': [1, 2, 4],
                          'random_forest__min_samples_split': [2, 5, 10],
                          'random_forest__n_estimators': [50, 100, 200, 500]}],
             scoring='f1')

In [468]:
print('Best hyperparameters:', grid_search_rnd_clf.best_params_)
print('Best score:', grid_search_rnd_clf.best_score_)

Best hyperparameters: {'random_forest__max_depth': 10, 'random_forest__max_features': 8, 'random_forest__min_samples_leaf': 4, 'random_forest__min_samples_split': 10, 'random_forest__n_estimators': 500}
Best score: 0.8090946061646336


In [469]:
cv_res = pd.DataFrame(grid_search_rnd_clf.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_random_forest__max_depth,param_random_forest__max_features,param_random_forest__min_samples_leaf,param_random_forest__min_samples_split,param_random_forest__n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
179,1.707157,0.066617,0.115277,0.008834,10,8,4,10,500,"{'random_forest__max_depth': 10, 'random_fores...",0.808922,0.809288,0.809074,0.809095,0.000150,1
166,0.722500,0.021955,0.051916,0.005420,10,8,2,10,200,"{'random_forest__max_depth': 10, 'random_fores...",0.809489,0.807178,0.808885,0.808517,0.000978,2
195,2.097208,0.042507,0.124085,0.013965,10,10,2,2,500,"{'random_forest__max_depth': 10, 'random_fores...",0.808543,0.807298,0.809288,0.808376,0.000821,3
119,1.488755,0.032954,0.133934,0.017319,10,6,1,10,500,"{'random_forest__max_depth': 10, 'random_fores...",0.809774,0.804314,0.810458,0.808182,0.002749,4
149,0.377332,0.008996,0.033220,0.011683,10,8,1,5,100,"{'random_forest__max_depth': 10, 'random_fores...",0.807319,0.805492,0.811450,0.808087,0.002492,5


### GridSearch CV - SVC

In [500]:
full_pipeline = Pipeline([
    ("preprocessing", preprocessing),
    ("svc", SVC(kernel="rbf", probability=True, random_state=42))
])

param_grid = [
    {
    'svc__gamma': [0.005, 0.01, 0.5, 0.8, 1, 1.5],
    'svc__C': [1, 3, 5, 10, 15],
    'svc__coef0': [0.5, 1, 1.5, 2],
    'svc__degree': [3, 5]
    }
]
grid_search_svc = GridSearchCV(full_pipeline, param_grid, cv=3,
                           scoring="f1", #"roc_auc", 
                           n_jobs=-1)
grid_search_svc.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standardize',
                                                                                          StandardScaler())]),
                                                                         Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('Fill '
                                                                                          'N/A '
                                                                                          'value',
                                                                                          SimpleImputer(s...
                                                                                          'Encoder',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         Index(['HomePlanet', 'CryoSleep', 'Destination', 'VIP', 'Carbin_deck',
       'Carbin_side'],
      dtype='object'))])),
                                       ('svc',
                                        SVC(probability=True,
                                            random_state=42))]),
             n_jobs=-1,
             param_grid=[{'svc__C': [1, 3, 5, 10, 15],
                          'svc__coef0': [0.5, 1, 1.5, 2],
                          'svc__degree': [3, 5, 8, 10],
                          'svc__gamma': [0.005, 0.01, 0.5, 0.8, 1, 1.5]}],
             scoring='f1')

In [504]:
print('Best hyperparameters:', grid_search_svc.best_params_)
print('Best score:', grid_search_svc.best_score_)

Best hyperparameters: {'svc__C': 15, 'svc__coef0': 0.5, 'svc__degree': 3, 'svc__gamma': 0.01}
Best score: 0.8072825908005891


In [505]:
cv_res = pd.DataFrame(grid_search_svc.cv_results_)
cv_res.sort_values(by='mean_test_score', ascending=False, inplace=True)
cv_res.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_svc__C,param_svc__coef0,param_svc__degree,param_svc__gamma,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
433,2.512490,0.047219,0.320935,0.036579,15,1.5,3,0.01,"{'svc__C': 15, 'svc__coef0': 1.5, 'svc__degree...",0.807918,0.798785,0.815145,0.807283,0.006694,1
403,2.528817,0.105091,0.321221,0.029398,15,0.5,10,0.01,"{'svc__C': 15, 'svc__coef0': 0.5, 'svc__degree...",0.807918,0.798785,0.815145,0.807283,0.006694,1
469,2.551950,0.110497,0.337435,0.042503,15,2,8,0.01,"{'svc__C': 15, 'svc__coef0': 2, 'svc__degree':...",0.807918,0.798785,0.815145,0.807283,0.006694,1
463,2.509884,0.065540,0.351033,0.056765,15,2,5,0.01,"{'svc__C': 15, 'svc__coef0': 2, 'svc__degree':...",0.807918,0.798785,0.815145,0.807283,0.006694,1
451,2.451157,0.060659,0.330120,0.007201,15,1.5,10,0.01,"{'svc__C': 15, 'svc__coef0': 1.5, 'svc__degree...",0.807918,0.798785,0.815145,0.807283,0.006694,1


## Stacking

In [471]:
estimators = [
    ('rnd_best_clf', RandomForestClassifier(max_depth=10, max_features=10, min_samples_leaf=4,
                                       min_samples_split=10, n_estimators =100,
                                       random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),
    ('rnd_clf', RandomForestClassifier(random_state=42))
]

stacking_classifer = StackingClassifier(
    estimators=estimators,
    final_estimator=RandomForestClassifier(random_state=43),
    cv=5,
    n_jobs=-1
)
stacking_clf = Pipeline([
    ('preprocessing', preprocessing),
    ('stacking_classifier', stacking_classifer)
])
stacking_clf.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardize',
                                                                   StandardScaler())]),
                                                  Index(['Age', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('Fill N/A '
                                                                   'value',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('One H...
                ('stacking_classifier',
                 StackingClassifier(cv=5,
                                    estimators=[('rnd_best_clf',
                                                 RandomForestClassifier(max_depth=10,
                                                                        max_features=10,
                                                                        min_samples_leaf=4,
                                                                        min_samples_split=10,
                                                                        random_state=42)),
                                                ('svc',
                                                 SVC(probability=True,
                                                     random_state=42)),
                                                ('rnd_clf',
                                                 RandomForestClassifier(random_state=42))],
                                    final_estimator=RandomForestClassifier(random_state=43),
                                    n_jobs=-1))])

# Evaluate the performance

## Evaluate the performance on the training set

In [472]:
# Randome Forest Tree
y_proba = rnd_clf.predict_proba(X_train)[:, 1]
y_pred = rnd_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9884399689062922
F1 score: 0.9496152891536361


In [473]:
# SVC
y_proba = svc_clf.predict_proba(X_train)[:, 1]
y_pred = svc_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8914970592757143
F1 score: 0.8162854988690625


In [506]:
# SVC Grid Search CV
y_proba = grid_search_svc.predict_proba(X_train)[:, 1]
y_pred = grid_search_svc.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8903590876146013
F1 score: 0.8117851815017405


In [474]:
# Randome Forest Tree Grid Search CV
y_proba = grid_search_rnd_clf.predict_proba(X_train)[:, 1]
y_pred = grid_search_rnd_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.9303188621564578
F1 score: 0.8432336990793291


In [475]:
# Stacking
y_proba = stacking_clf.predict_proba(X_train)[:, 1]
y_pred = stacking_clf.predict(X_train)
auc_roc = roc_auc_score(y_train, y_proba)
f1 = f1_score(y_train, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)


AUC-ROC score: 0.8910651477500207
F1 score: 0.7983921617887199


### Analyse Features

In [476]:
grid_search_model = grid_search_rnd_clf.best_estimator_
# feature_importances = grid_search_model
feature_importances = grid_search_model["random_forest"].feature_importances_
feature_importances.round(4)

sorted(zip(feature_importances, grid_search_model["preprocessing"].get_feature_names_out()), reverse=True)[:20]

[(0.1504419045484588, 'cat__CryoSleep_False'),
 (0.13040094896601986, 'cat__CryoSleep_True'),
 (0.12858975127543437, 'num__Spa'),
 (0.10893601993475205, 'num__VRDeck'),
 (0.10275947359440693, 'num__RoomService'),
 (0.08442383621105404, 'num__FoodCourt'),
 (0.06818086014424678, 'num__Age'),
 (0.05825890378285506, 'num__ShoppingMall'),
 (0.034800782831248066, 'cat__HomePlanet_Earth'),
 (0.02375873265098228, 'cat__HomePlanet_Europa'),
 (0.01895184426633816, 'cat__Carbin_deck_G'),
 (0.015053830248585398, 'cat__Carbin_deck_E'),
 (0.012673672694628761, 'cat__Carbin_deck_F'),
 (0.011752614697996908, 'cat__Carbin_side_S'),
 (0.011629002312588378, 'cat__Carbin_side_P'),
 (0.008504982183272454, 'cat__HomePlanet_Mars'),
 (0.006576072904889404, 'cat__Carbin_deck_C'),
 (0.006377792887565474, 'cat__Destination_55 Cancri e'),
 (0.006017237485644968, 'cat__Destination_TRAPPIST-1e'),
 (0.0040560975519441855, 'cat__Carbin_deck_B')]

## Evaluate the performance on the validation set

In [477]:
# randome forest tree
y_proba = rnd_clf.predict_proba(X_val)[:, 1]
y_pred = rnd_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8612372529221141
F1 score: 0.7735849056603775


In [478]:
# SVC
y_proba = svc_clf.predict_proba(X_val)[:, 1]
y_pred = svc_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.866114050993598
F1 score: 0.7910112359550563


In [507]:
# gridsearchCV - SVC
y_proba = grid_search_svc.predict_proba(X_val)[:, 1]
y_pred = grid_search_svc.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8673431627026713
F1 score: 0.7853170189098999


In [479]:
# gridsearchCV - Random Forest Tree
y_proba = grid_search_rnd_clf.predict_proba(X_val)[:, 1]
y_pred = grid_search_rnd_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)

AUC-ROC score: 0.8780298264441403
F1 score: 0.7867730900798175


In [480]:
# Stacking
y_proba = stacking_clf.predict_proba(X_val)[:, 1]
y_pred = stacking_clf.predict(X_val)
auc_roc = roc_auc_score(y_val, y_proba)
f1 = f1_score(y_val, y_pred)

# Print the AUC-ROC score
print('AUC-ROC score:', auc_roc)
print('F1 score:', f1)


AUC-ROC score: 0.8523665026088887
F1 score: 0.7736063708759955


# output data

In [481]:
# randome forest tree
y_test = rnd_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.7912

In [482]:
# SVC
y_test = svc_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.80406

In [508]:
# gridsearchCV - SVC
y_test = grid_search_svc.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.80173

In [483]:
# gridsearchCV - Random Forest Tree
y_test = grid_search_rnd_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.7877

In [484]:
# stacking
y_test = stacking_clf.predict(X_test)
output = pd.DataFrame(y_test, index=X_test.index, columns=['Transported'])
output.to_csv('submission.csv') # 0.77437

In [485]:
X_train.columns

Index(['HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP', 'RoomService',
       'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Carbin_deck',
       'Carbin_side'],
      dtype='object')